<a href="https://colab.research.google.com/github/yukti845/Reddit-Flair-Detection/blob/main/Web-Scrapping-and-Pre-Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install praw

     |████████████████████████████████| 153kB 8.1MB/s 
     |████████████████████████████████| 204kB 15.4MB/s 


In [2]:
import praw 
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import datetime as dt
nltk.download('all')
from bs4 import BeautifulSoup

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

### For getting the reddit data from praw we need to enter the credentials

In [3]:
reddit = praw.Reddit(client_id='TbfMVD4uOW0LDQ', client_secret='_YpcW2oj0l5GsI764j2XBOZB4hhRzg', user_agent='flair-detector', username='yukti_sachdeva', password='yuktisach31')

### Getting the subreddits and the topics and flairs

### labels_dict stores all the parts of reddit post that is flair, id, url, body, comments etc.
### flair contails all the list of flairs of the subreddit India.

In [4]:
subreddit = reddit.subreddit('india') 
labels_dict = {"flair":[], "title":[], "score":[], "id":[], "url":[], "comms_num":[], "created":[], "body":[], "author":[], "comments":[]}
flairs = ["AskIndia", "Non-Political", "[R]eddiquitte", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA", "Coronavirus"] 

### Appending the scrapped data into the dictionary

### We iterate through all the flairs and append the url, body, score, flairs etc in the dictionary.

In [5]:
for flair in flairs:
  get_subreddits = subreddit.search(flair, limit=100)

  for submission in get_subreddits:

    labels_dict["flair"].append(flair)
    labels_dict["title"].append(submission.title)
    labels_dict["score"].append(submission.score)
    labels_dict["id"].append(submission.id)
    labels_dict["url"].append(submission.url)
    labels_dict["comms_num"].append(submission.num_comments)
    labels_dict["created"].append(submission.created)
    labels_dict["body"].append(submission.selftext)
    labels_dict["author"].append(submission.author)

    submission.comments.replace_more(limit=None)
    comment=''
    for top_level_comment in submission.comments:
      comment = comment + ' ' + top_level_comment.body
    labels_dict["comments"].append(comment)

In [6]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)

In [7]:
data = pd.DataFrame(labels_dict)
time =data["created"].apply(get_date)
data =data.assign(timestamp = time)
del data['created']
data.to_csv('reddit-india-data.csv', index=False)

In [8]:

data = pd.read_csv('reddit-india-data.csv')
data.head()

,flair,title,score,id,url,comms_num,body,author,comments,timestamp
0,AskIndia,How to approach a girl?,2,k0qt2r,https://www.reddit.com/r/india/comments/k0qt2r...,13,2 years back I was working in a startup compan...,covidmanbun,Stop watching Indian movies. \nStop stalking...,2020-11-25 19:50:12
1,AskIndia,Where is gelatine available ?,0,jk9zlt,https://www.reddit.com/r/india/comments/jk9zlt...,4,I wish to buy gelatine and am looking for the ...,csstudentG,It's available in your regular kirana stores....,2020-10-29 21:26:38
2,AskIndia,Trevor Noah's jokes during Indo-Pak tensions a...,45,gv9lmh,https://www.reddit.com/r/india/comments/gv9lmh...,29,I don't really watch much of Trevor Noah's lat...,CommYouNitty,It's mostly because we as a country don't rea...,2020-06-02 23:42:24
3,AskIndia,"Buying used BS4 Scooty after April 1, Is there...",11,hpqq5o,https://www.reddit.com/r/india/comments/hpqq5o...,7,"I don't know if it's right place to ask, but ""...",akza07,Scammers will keep scamming. \n\nIt applies t...,2020-07-12 15:58:15
4,AskIndia,Need feedback for Insurance Policy that I took...,1,1s57oi,https://www.reddit.com/r/india/comments/1s57oi...,1,**Re-posting here because of lack of activity ...,dhavalcoholic,"Dear Policy Holder(Dhavalcoholic),\n \nWe req...",2013-12-05 14:30:23


In [9]:
def string(value):
  return str(value)

In [10]:
data['title'] = data['title'].apply(string)
data['body'] = data['body'].apply(string)
data['comments'] = data['comments'].apply(string)

In [11]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
def text_cleaning(text):

  text = BeautifulSoup(text, "lxml").text
  text = text.lower()
  text = REPLACE_BY_SPACE_RE.sub(' ',text)
  text = BAD_SYMBOLS_RE.sub(' ',text)
  text = ' '.join(word for word in text.split() if word not in STOPWORDS)
  return text

In [12]:
data['title'] = data['title'].apply(text_cleaning)
data['body'] = data['body'].apply(text_cleaning)
data['comments'] = data['comments'].apply(text_cleaning)

In [13]:
combined_features = data["title"] + data["comments"] + data["url"] + data["body"]
data = data.assign(combined_features = combined_features)

In [14]:
data.to_csv("data.csv",index = False)

In [15]:
data = pd.read_csv("data.csv")
data.head()

,flair,title,score,id,url,comms_num,body,author,comments,timestamp,combined_features
0,AskIndia,approach girl,2,k0qt2r,https://www.reddit.com/r/india/comments/k0qt2r...,13,2 years back working startup company pretty de...,covidmanbun,stop watching indian movies stop stalking go p...,2020-11-25 19:50:12,approach girlstop watching indian movies stop ...
1,AskIndia,gelatine available,0,jk9zlt,https://www.reddit.com/r/india/comments/jk9zlt...,4,wish buy gelatine looking cheapest local price...,csstudentG,available regular kirana stores usually kept r...,2020-10-29 21:26:38,gelatine availableavailable regular kirana sto...
2,AskIndia,trevor noah jokes indo pak tensions seriousnes...,45,gv9lmh,https://www.reddit.com/r/india/comments/gv9lmh...,29,really watch much trevor noah late night show ...,CommYouNitty,mostly country really matter lives pretty chea...,2020-06-02 23:42:24,trevor noah jokes indo pak tensions seriousnes...
3,AskIndia,buying used bs4 scooty april 1 issues registra...,11,hpqq5o,https://www.reddit.com/r/india/comments/hpqq5o...,7,know right place ask askindia asking saw artic...,akza07,scammers keep scamming applies new vehicles bu...,2020-07-12 15:58:15,buying used bs4 scooty april 1 issues registra...
4,AskIndia,need feedback insurance policy took x post ask...,1,1s57oi,https://www.reddit.com/r/india/comments/1s57oi...,1,posting lack activity r askindia hello last ye...,dhavalcoholic,dear policy holder dhavalcoholic request help ...,2013-12-05 14:30:23,need feedback insurance policy took x post ask...
